In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from math import sqrt

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Bidirectional, LSTM, concatenate
from tensorflow.keras.models import Model
# from sklearn import preprocessing
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import mean_absolute_error
# import numpy as np
# from tensorflow.keras import Input
# from tensorflow.keras.layers import Dense, LSTM
# from tensorflow.keras.models import load_model, Model

# import datetime
# from sklearn.metrics import mean_squared_error
# from tensorflow.keras.optimizers import Adam
# import matplotlib.pyplot as plt
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler
# from matplotlib import pyplot as plt

# from keras.models import Sequential
# from keras.layers import Dense, Dropout, LSTM, BatchNormalization
# from keras.layers.convolutional import Conv1D    
# from keras.layers import InputLayer, Conv1D, Dense, Flatten, MaxPooling1D,Bidirectional

# from keras.callbacks import TensorBoard
# from keras.callbacks import ModelCheckpoint
# from sklearn.metrics import mean_squared_error
# from keras.models import Sequential
# from tensorflow.keras.optimizers import SGD
# from matplotlib import pyplot

df_14, df_15, df_16, df_17, df_18, df_19, df_20 = [pd.read_csv(fr"../../../Taipei_{i}.csv") 
                                                   for i in range(14,21)]


# In[3]:


def get_X_and_Y(table, station_name = 'Banqiao'):
    table = table[table.SiteEngName =='Banqiao']
    features=table[['AMB_TEMP', 'CO', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'PM2.5','RAINFALL', 'RH']]
    #print(f'The shape of the input table is {features.shape}')
    data = np.array(features.values.reshape((-1,10*1)))
    timestep=16
    x_build = []
    for i in range(data.shape[0] - timestep - timestep):
        x_build.append(data[i:i+timestep])
    train_x = np.array(x_build)
    #print(f'The shape of the input train_x is {train_x.shape}')


    data = np.array(table['PM2.5'].values.reshape((-1, 1)))
    y_build = []
    for i in range(timestep, data.shape[0] - timestep):
        y_build.append(data[i:i+timestep])
    train_y = np.array(y_build)
    print(f'The shape of the input train_y is {train_y.shape}')
    print('-'*50)
    return (train_x, train_y)

train14_x, train14_y = get_X_and_Y(df_14, station_name = 'Banqiao')
train15_x, train15_y = get_X_and_Y(df_15, station_name = 'Banqiao')
train16_x, train16_y = get_X_and_Y(df_16, station_name = 'Banqiao')
train17_x, train17_y = get_X_and_Y(df_17, station_name = 'Banqiao')
train18_x, train18_y = get_X_and_Y(df_18, station_name = 'Banqiao')
train19_x, train19_y = get_X_and_Y(df_19, station_name = 'Banqiao')
train20_x, train20_y = get_X_and_Y(df_20, station_name = 'Banqiao')


# In[16]:


train_X=np.concatenate((train14_x,train15_x,train16_x,train17_x,train18_x,train19_x),axis=0)
#print(train_X.shape)
train_y=np.concatenate((train14_y,train15_y,train16_y,train17_y,train18_y,train19_y),axis=0)
#print(train_y.shape)
test_X=train20_x;test_y=train20_y
#print(train20_x.shape,train20_y.shape)


from sklearn.model_selection import train_test_split
# create dataset
X, y = test_X,test_y

# split into train test sets
valid_x,test_x,valid_y,test_y = train_test_split(X, y, test_size=0.5)
print(valid_x.shape, valid_y.shape,test_x.shape, test_y.shape)

print("train_X.shape, train_y.shape, test_x.shape, test_y.shape") 
print(train_X.shape, train_y.shape, test_x.shape, test_y.shape) 

####################################
######################################


# Define the input layers
input1 = Input(shape=(train_X.shape[1], train_X.shape[2]))
x1 = Conv1D(filters=100, kernel_size=1, activation='relu')(input1)
x1 = MaxPooling1D(pool_size=1)(x1)
lstm1 = Bidirectional(LSTM(50, return_sequences=True))(x1)
output1 = Dense(50, activation='relu')(lstm1)
out=Dense(1, activation='relu')(output1)
###########################
model = Model(inputs=[input1], outputs=out)

model.compile(loss='mse',optimizer='adam',metrics=['mae'])

model.summary()

# # Compile the model
# model.compile(loss='mse',optimizer='adam',metrics=['mae'])

# # Print model summary
# model.summary()

The shape of the input train_y is (8728, 16, 1)
--------------------------------------------------
The shape of the input train_y is (8728, 16, 1)
--------------------------------------------------
The shape of the input train_y is (8728, 16, 1)
--------------------------------------------------
The shape of the input train_y is (8728, 16, 1)
--------------------------------------------------
The shape of the input train_y is (8728, 16, 1)
--------------------------------------------------
The shape of the input train_y is (8728, 16, 1)
--------------------------------------------------
The shape of the input train_y is (4292, 16, 1)
--------------------------------------------------
(2146, 16, 10) (2146, 16, 1) (2146, 16, 10) (2146, 16, 1)
train_X.shape, train_y.shape, test_x.shape, test_y.shape
(52368, 16, 10) (52368, 16, 1) (2146, 16, 10) (2146, 16, 1)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape          

In [3]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import r2_score, mean_absolute_percentage_error
import numpy as np
checkpoint_path = "Banqiao161.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback   = [
      EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.0001),
      ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')
]

#Train the model with the new callback
history = model.fit((train_X) ,
                    (train_y), 
                    validation_data=((valid_x),
                                    (valid_y)),
                    epochs =50, batch_size=19, verbose = 2, shuffle = True,
                    callbacks=[cp_callback])


# # In[ ]:


# model.load_weights(checkpoint_path)


# In[ ]:


scores = model.evaluate(test_x, test_y, verbose=0)
print('Test loss:', scores)
print('Test accuracy:', scores)
predict_ary = model.predict([test_x.astype("float")], batch_size = 190)
print (predict_ary.shape)
print (test_y.shape)
#print (X_valid.shape)
rmse_score20 = np.sqrt(np.mean(np.square(predict_ary - test_y.astype("float"))))
mae_score = np.mean(np.abs(predict_ary - test_y.astype("float")))
#mape_score = mean_absolute_percentage_error(y_valid_c.astype("float"),predict_ary)
#mae2 = mean_absolute_error(predict_ary, validation_Y[:-3])
print('this is rmse ',rmse_score20)
#print('this is mape ',mape_score)
print('this is mae ',mae_score)

y_pred_flat = predict_ary.reshape(-1)
y_true_flat = test_y.astype("float").reshape(-1)

# Calculate metrics
r2 = r2_score(y_true_flat, y_pred_flat)
mape = mean_absolute_percentage_error(y_true_flat, y_pred_flat)
print('this is r2 ',r2)
print('this is mape ',mape)
# In[ ]:


Epoch 1/50
2757/2757 - 22s - loss: 98.3434 - mae: 6.9631 - val_loss: 49.2246 - val_mae: 5.2123 - 22s/epoch - 8ms/step
Epoch 2/50
2757/2757 - 17s - loss: 82.8917 - mae: 6.3739 - val_loss: 51.3728 - val_mae: 5.2883 - 17s/epoch - 6ms/step
Epoch 3/50
2757/2757 - 16s - loss: 78.8068 - mae: 6.2147 - val_loss: 51.2091 - val_mae: 5.3592 - 16s/epoch - 6ms/step
Epoch 4/50
2757/2757 - 18s - loss: 74.9460 - mae: 6.0877 - val_loss: 55.0939 - val_mae: 5.7054 - 18s/epoch - 7ms/step
Epoch 5/50
2757/2757 - 17s - loss: 70.8187 - mae: 5.9533 - val_loss: 56.8479 - val_mae: 5.7374 - 17s/epoch - 6ms/step
Epoch 6/50
2757/2757 - 17s - loss: 65.3738 - mae: 5.7711 - val_loss: 70.2002 - val_mae: 6.3599 - 17s/epoch - 6ms/step
Epoch 7/50
2757/2757 - 18s - loss: 60.8322 - mae: 5.6127 - val_loss: 60.7192 - val_mae: 5.9233 - 18s/epoch - 7ms/step
Epoch 8/50
2757/2757 - 17s - loss: 56.5966 - mae: 5.4489 - val_loss: 67.1667 - val_mae: 6.2606 - 17s/epoch - 6ms/step
Epoch 9/50
2757/2757 - 18s - loss: 53.1959 - mae: 5.3143

In [4]:


#################
import csv
data = [[16,rmse_score20,mae_score,mape,r2,"Banqiao"]]
file = open('Banqiao.csv', 'a+', newline ='')

# writing the data into the file
with file:    
    write = csv.writer(file)
    write.writerows(data)
dfa47= pd.read_csv("Banqiao.csv")
dfa47







,HR,RMSE,MAE,MAPE,R2,Site
0,1,7.322181,5.464624,0.54855,0.401974,Banqiao
1,16,7.322181,5.464624,0.54855,0.401974,Banqiao


Test loss: [53.61433029174805, 5.464622497558594]
Test accuracy: [53.61433029174805, 5.464622497558594]
12/12 [==============================] - 0s 9ms/step
(2146, 16, 1)
(2146, 16, 1)
this is rmse  7.322180882441001
this is mae  5.464623854852457
this is mape  54.85504493770625
